## Import libraries

In [9]:
import ee
import geemap
import geemap.chart as ui_chart

## Create an interactive map

In [10]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [16]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F4.2 Aggregating Images for Time Series
#  Checkpoint:   F42b
#  Author:       Ujaval Gandhi
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')
year = 2019
startDate = ee.Date.fromYMD(year, 1, 1)

endDate = startDate.advance(1, 'year')

yearFiltered = chirps \
.filter(ee.Filter.date(startDate, endDate))
print(yearFiltered.getInfo(), 'Date-filtered CHIRPS images')

print(startDate.getInfo(), 'Start date')
print(endDate.getInfo(), 'End date')

print('Start date as timestamp', startDate.millis().getInfo())
print('End date as timestamp', endDate.millis().getInfo())

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

# Aggregate this time series to compute monthly images.
# Create a list of months
months = ee.List.sequence(1, 12)

# Write a function that takes a month number
# and returns a monthly image.

def func_lcx(beginningMonth):
    startDate = ee.Date.fromYMD(year, beginningMonth, 1)
    endDate = startDate.advance(1, 'month')
    monthFiltered = yearFiltered \
    .filter(ee.Filter.date(startDate, endDate))

    # Calculate total precipitation.
    total = monthFiltered.reduce(ee.Reducer.sum())
    return total.set({
        'system:time_start': startDate.millis(),
        'system:time_end': endDate.millis(),
        'year': year,
        'month': beginningMonth
        })

createMonthlyImage = func_lcx















# map() the function on the list of months
# This creates a list with images for each month in the list
monthlyImages = months.map(createMonthlyImage)

# Create an ee.ImageCollection.
monthlyCollection = ee.ImageCollection.fromImages(monthlyImages)
print(monthlyCollection.getInfo())

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

# Create a point with coordinates for the city of Bengaluru, India.
point = ee.Geometry.Point(77.5946, 12.9716)

chart = ui_chart.image_series(
image_collection = monthlyCollection,
region = point,
reducer = ee.Reducer.mean(),
scale = 5566,
)
chart.display()

chart = ui_chart.image_series(
image_collection = monthlyCollection,
region = point,
reducer = ee.Reducer.mean(),
scale = 5566)
chart.set_options(options = {
    'lineWidth': 1,
    'pointSize': 3,
    'title': 'Monthly Rainfall at Bengaluru',
    'vAxis': {
        'title': 'Rainfall (mm)'
        },
    'hAxis': {
        'title': 'Month',
        'gridlines': {
                'count': 12
            }
        }
    })
chart.display()

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------


{'type': 'ImageCollection', 'bands': [], 'version': 1724452176693717, 'id': 'UCSB-CHG/CHIRPS/PENTAD', 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'keywords': ['chg', 'climate', 'geophysical', 'precipitation', 'ucsb', 'weather'], 'thumb': 'https://mw1.google.com/ges/dd/images/CHIRPS_thumb.png', 'description': '<p>Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS)\nis a 30+ year quasi-global rainfall dataset. CHIRPS incorporates\n0.05° resolution satellite imagery with in-situ station data\nto create gridded rainfall time series for trend analysis and seasonal\ndrought monitoring.</p><p><b>Provider: <a href="https://chc.ucsb.edu/data/chirps">UCSB/CHG</a></b><br><p><b>Resolution</b><br>5566 meters\n</p><p><b>Cadence</b><br>\n    1\n  pentad\n<br>Each asset spans a pentad. Each of first 5 pentads in a month\nhave 5 days. The last pentad contains all the days from the 26th to the\nend of the month.\n</p><p><b>Bands</b><table class

Figure(axes=[Axis(label_offset='30px', scale=DateScale()), Axis(label_offset='40px', orientation='vertical', s…

Output(layout=Layout(width='50%'))

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, scale_x=LinearScale(allow_padding=False,…

## Display the interactive map

In [12]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…